# Integration with slide level-labels

In this tutorial we will demonstrate how to integrate whole slide images (WSIs) with slide-level labels and derive quantitative scores for each slide via top-K scoring.

We will also demonstrate how to run tasks in a distributed fashion using dask.

For this, we will be using a pre-processed dataset of artery tissue from GTEx, whihc contains healthy and calcified samples.

In [2]:
from huggingface_hub import hf_hub_download
import pandas as pd

table = hf_hub_download(
    "rendeirolab/lazyslide-data", 
    "GTEx_artery_dataset.csv.gz",
    repo_type="dataset"
)

dataset = pd.read_csv(table)
dataset.head()

,Tissue Sample Id,Sex,Age Bracket,Pathology Categories
0,GTEX-111YS-2226,male,60-69,calcification
1,GTEX-11GSP-2926,female,60-69,calcification
2,GTEX-11LCK-1426,male,30-39,clean_specimens
3,GTEX-11ONC-2726,male,60-69,calcification
4,GTEX-12126-0726,male,20-29,clean_specimens


Since we need to run for many slides, let's first define a function to process a slide and reuse it.

In [ ]:
from wsidata import open_wsi
import lazyslide as zs
import numpy as np
import pooch

In [ ]:
terms = ["calcification", "atherosclerosis"]

def wsi_feature_extraction(slide):
    s = pooch.retrieve(
        url=f"https://brd.nci.nih.gov/brd/imagedownload/{slide}",
        fname=f"{slide}.svs",
        path="gtex_artery_slides",
        known_hash=None,
    )
    wsi = open_wsi(s)
    zs.pp.find_tissues(wsi)
    zs.pp.tile_tissues(wsi, 256, mpp=0.5, background_fraction=0.5)

    # conch feature
    zs.tl.feature_extraction(wsi, "conch", pbar=False)
    embed = zs.tl.text_embedding(terms, "conch")
    zs.tl.text_image_similarity(wsi, embed, "conch")
    wsi.write()

    # score the slide with Top-K max pooling
    scores = zs.metrics.topk_scores(wsi["conch_tiles_text_similarity"], k=100)

    return scores

## Run for every slides

The easist way is to run a for-loop:

```python
for slide in dataset["Tissue Sample Id"]:
    wsi_feature_extraction(slide)
```

However, this will take a long time and doesn't fully use the power of parallism.

## Distributed processing with dask

Dask is a good option for parallism on local machine or across multiple machines.

For different hardward availabilities, alternatives are:
1. [dask-jobqueue](https://jobqueue.dask.org/en/latest/): For PBS, Slurm, MOAB, SGE, LSF, and HTCondor.
2. [coiled](https://docs.coiled.io/user_guide/index.html): AWS, GCP, Azure etc.
3. [dask-cuda](https://docs.rapids.ai/api/dask-cuda/nightly/quickstart/): If you have multiple GPU cards locally.

Here, we showcase how to parallel the jobs with dask on a SLURM cluster.
The configuration may not work on your SLURM system, please make adjustment accordingly.

When running GPU-intensive works like feature extraction for multiple WSIs, 
we recommanded to run one task on one GPU every time.
To accelarate the processing speed, either distributed across multiple GPU cards or multiple machines.

In [ ]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client

cluster = SLURMCluster(
    queue="gpu",
    cores=8,
    processes=1,
    memory="20 GB",
    job_extra_directives=["-q gpu", "--gres=gpu:h100pcie:1", "--time=02:00:00"],
    worker_extra_args=["--resources GPU=1"],
    log_directory="./dask-logs",
)

client = Client(cluster)
cluster.scale(3)  # Request for 3 workers, each owns one H100

Let's parallel the jobs

In [39]:
futures = [
    client.submit(wsi_feature_extraction, slide, resources={"GPU": 1})
    for slide in dataset["Tissue Sample Id"]
]

If you want to monitor the process, you can either go to the dask dashboard or use a simple progress bar

In [ ]:
from dask.distributed import as_completed
from tqdm.auto import tqdm

for _ in tqdm(as_completed(futures), total=len(futures)):
    pass

Our function returns the scores for the two pathological terms that we defined, 
we can collect the scores and save it for further analysis.

In [41]:
slide_scores = pd.DataFrame(
    np.vstack([f.result() for f in futures]),
    columns=terms,
    index=dataset["Tissue Sample Id"],
)
slide_scores.to_csv("slide_scores.csv")

In [87]:
client.shutdown()

## Slide aggregation

After the slides are processed to have slide-level features and scores, we will aggregate them into an AnnData object.

In [43]:
from wsidata import agg_wsi

dataset["slide"] = [f"gtex_artery_slides/{s}.svs" for s in dataset["Tissue Sample Id"]]
agg_data = agg_wsi(dataset, "uni2", wsi_col="slide", agg_key="agg_slide")
agg_data.obs = agg_data.obs.merge(slide_scores, on="Tissue Sample Id").set_index(
    "Tissue Sample Id"
)
agg_data

AnnData object with n_obs × n_vars = 45 × 1536
    obs: 'Sex', 'Age Bracket', 'Pathology Categories', 'slide', 'calcification', 'atherosclerosis'

In [44]:
agg_data.write_h5ad("agg_uni2_features.h5ad")